# Récupération des votes des député.e.s de l'Assemblée nationale

Afin d'analyser les votes des député.e.s siégants à l'Assemblée nationale, nous commençons par récupérer un fichier csv contenant l'ensemble des scrutins de la XVIème législature en passant par le lien suivant : https://data.assemblee-nationale.fr/travaux-parlementaires/votes.

Nous utilisons également dans le code suivant pour obtenir la liste des député.e.s en exercice disponible via le lien suivant : https://data.assemblee-nationale.fr/acteurs/deputes-en-exercice.

Cela nous a permis de récupérer 1351 scrutins le 31 mars 2023.

## 1. Liste des bibliothèques utilisées

In [45]:
import json
import pandas as pd
import os

## 2. Création d'un path finder et extraction des votes des député.e.s aux différents scrutins

Il y a quatre votes possibles pour chaque scrutin : "non Votant", "pour", "contre, "abstention".

In [1]:
votes_possibles = ["nonVotants", "pours", "contres", "abstentions"]

Le fichier que nous avons obtenu depuis le site de l'Assemblée nationale est de la forme suivante : un dossier contenant un fichier json pour chaque scutin. Les fichiers jsons n'ont pas toujours exactement la même structure. Il faut donc tout d'abords créer une fonction de recherche capable d'intéroger tout fichier json simple 'data' pour en extraire le chemin d'accès vers une valeur donnée 'value'. Ainsi, ces deux fonctions sont utilisées conjointement pour trouver un chemin d'accès dans un dictionnaire `data` menant à une valeur spécifique, et ensuite extraire une valeur spécifique de ce chemin d'accès en fonction de certains critères définis dans `get_path_value`.

In [46]:
def find_path(data, value, path=None):
    """
    Parcourir récursivement le dictionnaire 'data' et retourner le chemin d'accès de la première occurrence de 'value'.
    """
    if path is None:
        path = []
    if isinstance(data, dict):
        for k, v in data.items():
            new_path = path + [k]
            if v == value:
                return new_path
            elif isinstance(v, dict) or isinstance(v, list):
                result = find_path(v, value, new_path)
                if result is not None:
                    return result
    elif isinstance(data, list):
        for i, v in enumerate(data):
            new_path = path + [i]
            if v == value:
                return new_path
            elif isinstance(v, dict) or isinstance(v, list):
                result = find_path(v, value, new_path)
                if result is not None:
                    return result
    return None

def get_path_value(path):
    if path is not None:
        for element in path:
            if element in votes_possibles:
                return element
    return None

## Application des fonctions find_path et get_path_value à l'ensemble des scrutins

Après avoir créé nos fonctions de recherche, nous pouvons les appliquer à notre dossier issu du site de l'Assemblée nationale. Pour cela, nous commençons par créer un dataframe df à partir de la liste des députés et effectuons un peu de nettoyage de la base. Après, nous créons une variable répertoire où nous mettons le chemin vers le dossier contenant les scrutins.

In [ ]:
# importer la liste des députés en exercice
df = pd.read_csv("data\\20230430_Data_Depute_Liste.csv", sep=";", encoding='ISO-8859-1')
df['identifiant'] = df['identifiant'].apply(lambda x: 'PA' + str(x))

# Chemin d'accès au répertoire contenant les fichiers JSON
repertoire = 'data\\20230331_Data_VotesDeputesJson'

# Liste des fichiers JSON dans le répertoire
fichiers = os.listdir(repertoire)

Ensuite et de façon itérative, nous analysons l'ensemble des fichiers du répertoire à la recherche de l'identifiant de chaque député pour obtenir via son path son vote lors de chaque scrutin.

In [ ]:
i=0

# Itération sur les fichiers JSON
for nom_du_fichier in fichiers:
    # Chemin d'accès complet au fichier JSON
    chemin_fichier = os.path.join(repertoire, nom_du_fichier)

    # Chargement des données JSON
    with open(chemin_fichier) as f:
        data = json.load(f)
    
    # Extrait le nom du fichier à partir du chemin
    nom_scrutin = os.path.basename(nom_du_fichier)

    # Supprime l'extension ".json" du nom de fichier
    scrutin = nom_scrutin.rstrip('.json')

    # Créer une nouvelle colonne path initialisée à None
    df[str(scrutin)] = None

    # Appliquer la fonction get_path_value à chaque ligne du DataFrame
    df[str(scrutin)] = df.apply(lambda row: get_path_value(find_path(data, row['identifiant'], path=None)), axis=1)
    
    i += 1
    print(i)

df


In [48]:
df.to_csv('20230430_Data_Votes_Deputes.csv', index=False)